# Lab 7 - Bonus labs

## 7.1 - Time travel

In [ ]:
-- Step 7.1.3

SELECT 'dbo.FactSale'   AS TableName, COUNT(*) AS RecordCount FROM dbo.FactSale   WHERE InvoiceDate = '2013-01-01'
ORDER BY TableName DESC

In [ ]:
-- Step 7.1.4

DELETE FROM dbo.FactSale WHERE InvoiceDate = '2013-01-01'
SELECT COUNT(*) AS RecordCount FROM dbo.FactSale WHERE InvoiceDate = '2013-01-01'

In [ ]:
-- Step 7.1.5

DECLARE @BeforeTheAccident VARCHAR(23) = (SELECT CONVERT(VARCHAR(23), DATEADD(MINUTE, -30, GETDATE()), 126))
EXEC ('SELECT COUNT(*) FROM dbo.FactSale WHERE InvoiceDate = ''2013-01-01'' OPTION (FOR TIMESTAMP AS OF ''' + @BeforeTheAccident + ''')');

### 7.2 - Clone a table

In [ ]:
-- Step 7.2.1

DROP TABLE IF EXISTS dbo.FactSale_Recovery

DECLARE @BeforeTheAccident VARCHAR(23) = (SELECT CONVERT(VARCHAR(23), DATEADD(MINUTE, -30, GETDATE()), 126))

EXEC ('CREATE TABLE dbo.FactSale_Recovery AS CLONE OF dbo.FactSale AT ''' + @BeforeTheAccident + '''')

In [ ]:
-- Step 7.2.2

SELECT 'dbo.FactSale'          AS TableName, COUNT(*) AS RecordCount FROM dbo.FactSale          WHERE InvoiceDate = '2013-01-01' UNION ALL
SELECT 'dbo.FactSale_Recovery' AS TableName, COUNT(*) AS RecordCount FROM dbo.FactSale_Recovery WHERE InvoiceDate = '2013-01-01'
ORDER BY TableName DESC

In [ ]:
-- Step 7.2.3

INSERT INTO dbo.FactSale
SELECT * FROM dbo.FactSale_Recovery WHERE InvoiceDate = '2013-01-01'

SELECT 'dbo.FactSale'          AS TableName, COUNT(*) AS RecordCount FROM dbo.FactSale          WHERE InvoiceDate = '2013-01-01' UNION ALL
SELECT 'dbo.FactSale_Recovery' AS TableName, COUNT(*) AS RecordCount FROM dbo.FactSale_Recovery WHERE InvoiceDate = '2013-01-01'
ORDER BY TableName DESC

## 7.3 - Custom data masking

In [ ]:
-- Step 7.3.1
CREATE OR ALTER FUNCTION dbo.MaskWebsiteURL (@url VARCHAR(256))
RETURNS VARCHAR(256)
AS
BEGIN
    DECLARE @result VARCHAR(256);

    -- Check if the URL is valid using a simple pattern matching for 'http://' or 'https://'
    IF PATINDEX('http://%', @url) = 1 OR PATINDEX('https://%', @url) = 1
    BEGIN
        -- Apply masking after 'http://' or 'https://'
        -- Find the first '/' after the domain part (to keep the domain visible)
        DECLARE @firstSlashPos INT;
        SET @firstSlashPos = CHARINDEX('/', @url, CHARINDEX('://', @url) + 3);

        -- If a slash is found, mask from there, otherwise just return the original URL
        IF @firstSlashPos > 0
        BEGIN
            SET @result = STUFF(@url, @firstSlashPos + 1, LEN(@url), REPLICATE('*', LEN(@url) - @firstSlashPos));
        END
        ELSE
        BEGIN
            -- Masking does not apply here, without a path
            SET @result = @url;
        END
    END
    ELSE
    BEGIN
        SET @result = 'invalid url';
    END

    RETURN @result;
END;
GO

In [ ]:
-- Step 7.3.2
CREATE OR ALTER FUNCTION dbo.MaskPhoneNumber (@phoneNumber VARCHAR(20))
RETURNS VARCHAR(20)
AS
BEGIN
    DECLARE @result VARCHAR(20);

    -- Check if the phone number matches the expected format (e.g., (210) 555-0100)
    IF PATINDEX('(%[0-9][0-9][0-9]%) [0-9][0-9][0-9]-[0-9][0-9][0-9][0-9]', @phoneNumber) = 1
    BEGIN
        -- Mask the phone number, only revealing the area code and the last four digits
        SET @result = STUFF(@phoneNumber, 7, 3, '***');
    END
    ELSE
    BEGIN
        SET @result = 'invalid phone';
    END

    RETURN @result;
END;
GO

In [ ]:
-- Step 7.3.3
SELECT
    OBJECT_SCHEMA_NAME(sm.object_id) AS objectschemaname,
    OBJECT_NAME(sm.object_id) AS objectname,
    sm.definition,
    sm.is_inlineable
FROM sys.sql_modules sm
WHERE OBJECTPROPERTY(sm.object_id, 'IsScalarFunction')=1

In [ ]:
-- Step 7.3.4
CREATE OR ALTER   VIEW dbo.vw_DimCustomer AS
SELECT
            [CustomerSK],
			[CustomerSourceKey],
			[ContactFirstName],
			[ContactLastName],
			dbo.MaskPhoneNumber([PhoneNumber]) as PhoneNumber,
			dbo.MaskWebsiteURL([WebsiteURL]) AS WebsiteURL,
			[DeliveryAddressLine],
			[DeliveryPostalCode],
			[TimeInserted],
			[TimeLastChanged]
FROM dbo.DimCustomer
WHERE IsDeleted = 0;

In [ ]:
-- Step 7.3.5
SELECT TOP 10 *
from dbo.vw_DimCustomer 

## 7.4 - Change existing dimensional model

In [ ]:
-- 7.4.1 Data expoloration: Check current data distribution
SELECT  Tags, COUNT(*) AS Frequency
FROM dbo.DimStockItem
GROUP BY Tags
ORDER BY Frequency DESC

In [ ]:
-- 7.4.2 Data expoloration: Check can OPENJSON help to get desired distribution

SELECT value AS Tag, COUNT(*) AS Frequency
FROM dbo.DimStockItem
CROSS APPLY OPENJSON(Tags)
GROUP BY value
ORDER BY Frequency DESC


In [ ]:
-- 7.4.3 Add new table and columns

DROP TABLE IF EXISTS dbo.DimStockItemTag;
GO

-- Create DimStockItemTag table
CREATE TABLE [dbo].[DimStockItemTag]
    (
        [StockItemTagSK]                  [int]           NOT NULL,
        [StockItemTagName]                [varchar](250)  NOT NULL,
        [TimeInserted]                    [datetime2](6)  NOT NULL,
        [TimeLastChanged]                 [datetime2](6)  NOT NULL,
        [IsDeleted]                       [bit]           NOT NULL
    );
GO
-- Add Tag ID in DimStockItem table. Notice it NULLable
IF NOT EXISTS (
    SELECT 1 
    FROM INFORMATION_SCHEMA.COLUMNS 
    WHERE TABLE_SCHEMA = 'dbo' 
      AND TABLE_NAME = 'DimStockItem' 
      AND COLUMN_NAME = 'StockItemTagSK'
)
BEGIN
    ALTER TABLE DimStockItem ADD StockItemTagSK INT;
END
GO



In [ ]:
-- 7.4.4 Confirm that new table and new column has been created (add instructions to Lab)
SELECT TOP 10 *
FROM [dbo].DimStockItem
GO
SELECT TOP 10 *
FROM [dbo].DimStockItemTag
GO

In [ ]:
-- 7.4.5: Add Unknown Member and Adjust keys

IF NOT EXISTS (SELECT * FROM dbo.DimStockItemTag WHERE StockItemTagSK = -1)
INSERT INTO [dbo].[DimStockItemTag]
           ([StockItemTagSK]
           ,[StockItemTagName]
           ,[TimeInserted]
           ,[TimeLastChanged]
           ,[IsDeleted]) 
SELECT -1, 'Unknown', GETUTCDATE(), GETUTCDATE(), 0
GO

UPDATE dbo.DimStockItem
SET StockItemTagSK=-1, TimeLastChanged=GETUTCDATE()
WHERE StockItemTagSK IS NULL
GO




In [ ]:
-- 7.4.6: Validate that new table has only Unknown member
SELECT TOP 10 *
FROM dbo.DimStockItemTag


In [ ]:
-- 7.4.7: Load DimStockItemTag table by using OPENJSON

------------------------------------- Load DimStockItemTag table by using OPENJSON  - ------------------------
DECLARE @MaxID INT = (SELECT ISNULL(MAX(StockItemTagSK), 0) FROM dbo.DimStockItemTag);
DECLARE @CurrentUTC DATETIME2= GETUTCDATE();

WITH DistinctTags AS (
        SELECT DISTINCT value AS StockItemTagName
        FROM WWI_Silver.dbo.StockItems
        CROSS APPLY OPENJSON(Tags)
        WHERE Tags IS NOT NULL
),
src AS (
    SELECT
        @MaxID + ROW_NUMBER() OVER (ORDER BY (SELECT NULL)) AS StockItemTagSK,
        StockItemTagName
    FROM DistinctTags
)
MERGE dbo.DimStockItemTag T
USING src AS S
    -- match on normalized business key
    ON T.StockItemTagName = S.StockItemTagName
WHEN MATCHED AND (
       -- update only if something actually changed (e.g., casing/spacing)
       ISNULL(T.StockItemTagName, '') <> ISNULL(S.StockItemTagName, '')
) THEN
    UPDATE SET
        T.StockItemTagName = S.StockItemTagName,
        T.TimeLastChanged=@CurrentUTC
WHEN NOT MATCHED BY TARGET THEN
    INSERT (StockItemTagSK, StockItemTagName, TimeInserted, TimeLastChanged, IsDeleted)
    VALUES (S.StockItemTagSK, S.StockItemTagName, @CurrentUTC, @CurrentUTC, 0);
GO


In [ ]:
-- 7.4.8: Confirm that DimStockItemTag is loaded
SELECT *
FROM dbo.DimStockItemTag
GO

In [ ]:
-- 7.4.9: Check cardinality of DimStockItem table
-- 228 records expected
SELECT COUNT(*)
FROM dbo.DimStockItem


In [ ]:
-- 7.4.10: (Re)Load new DimStockItem table structure

DECLARE @MaxID int = (SELECT ISNULL(MAX(StockItemSK), 0) FROM dbo.DimStockItem);
DECLARE @CurrentUTC DATETIME2= GETUTCDATE();

;WITH src AS
(
    SELECT DISTINCT
           StockItemSK = @MaxID + ROW_NUMBER() OVER (ORDER BY (SELECT NULL)),
           StockItemSourceKey = si.StockItemID,
           si.StockItemName,
           ISNULL(db.BrandSK, -1) AS BrandSK,
           si.PackageTypeName,
           si.TaxRate,
           si.UnitPrice,
           ISNULL(dtag.StockItemTagSK,-1) as StockItemTagSK
    FROM WWI_Silver.dbo.StockItems AS si
         LEFT JOIN dbo.DimBrand AS db
            ON db.BrandName = si.Brand
         OUTER APPLY OPENJSON(si.Tags) AS otag
         LEFT JOIN dbo.DimStockItemTag AS dtag 
            ON dtag.StockItemTagName=otag.value
)
MERGE dbo.DimStockItem T
USING (
    -- Align datatypes to target to ensure null-safe comparisons are accurate
    SELECT
        S.StockItemSK,
        S.StockItemSourceKey,
        S.StockItemName,
        CAST(S.BrandSK AS varchar(50)) AS BrandSK,  -- target is varchar(50)
        S.PackageTypeName,
        S.TaxRate,
        S.UnitPrice,
        S.StockItemTagSK as StockItemTagSK
    FROM src AS S
) AS S
ON T.StockItemSourceKey = S.StockItemSourceKey AND T.StockItemTagSK= S.StockItemTagSK

WHEN MATCHED 
AND (
       ISNULL(T.StockItemName,    '') <> ISNULL(S.StockItemName,    '')
    OR ISNULL(T.BrandSK,          '') <> ISNULL(S.BrandSK,          '')
    OR ISNULL(T.PackageTypeName,  '') <> ISNULL(S.PackageTypeName,  '')
    OR ISNULL(T.TaxRate,         0.0) <> ISNULL(S.TaxRate,         0.0)
    OR ISNULL(T.UnitPrice,       0.0) <> ISNULL(S.UnitPrice,       0.0)
    OR ISNULL(T.StockItemTagSK, -1)   <> ISNULL(S.StockItemTagSK, -1)
) THEN
    UPDATE SET
        T.StockItemName     = S.StockItemName,
        T.BrandSK           = S.BrandSK,
        T.PackageTypeName   = S.PackageTypeName,
        T.TaxRate           = S.TaxRate,
        T.UnitPrice         = S.UnitPrice,
        T.StockItemTagSK    = ISNULL(S.StockItemTagSK, -1),
        T.TimeLastChanged   = @CurrentUTC

WHEN NOT MATCHED BY TARGET THEN
    INSERT (
        StockItemSK,
        StockItemSourceKey,
        StockItemName,
        BrandSK,
        PackageTypeName,
        TaxRate,
        UnitPrice,
        StockItemTagSK,
        TimeInserted,
        TimeLastChanged,
        IsDeleted
    )
    VALUES (
        S.StockItemSK,
        S.StockItemSourceKey,
        S.StockItemName,
        S.BrandSK,
        S.PackageTypeName,
        S.TaxRate,
        S.UnitPrice,
        S.StockItemTagSK,
        @CurrentUTC,
        @CurrentUTC,
        0
    )
-- Delete old rows with NULL tag key
WHEN NOT MATCHED BY SOURCE 
    AND T.StockItemTagSK=-1 
    AND IsDeleted=0 
    AND StockItemSK!=-1 -- keep uknown member
    THEN 
    UPDATE SET IsDeleted=1;
GO


In [ ]:
-- 7.4.11: Confirm that table cardinality has increased and expected data is loaded
-- Check IsDeleted distribution
SELECT IsDeleted, COUNT(*) as Frequency
FROM dbo.DimStockItem
GROUP BY IsDeleted
ORDER BY 1
GO
-- Check those with multiple entries per BK (contain multiple tags)
SELECT StockItemSourceKey, COUNT(*) as Frequency
FROM dbo.DimStockItem
GROUP BY StockItemSourceKey
HAVING COUNT(*)>1
ORDER BY 1, 2 DESC
GO
-- Check StockItemTagSK distribution
SELECT IsDeleted, StockItemTagSK, COUNT(*) as Frequency
FROM dbo.DimStockItem
GROUP BY IsDeleted, StockItemTagSK
ORDER BY 1, 2 DESC
GO
-- Explore data
SELECT TOP 10 *
FROM dbo.DimStockItem
WHERE StockItemSourceKey=10
GO

In [ ]:
-- 7.4.12: Drop Tags column from DimStockItem table by using ALTER table

IF EXISTS (
    SELECT 1 
    FROM INFORMATION_SCHEMA.COLUMNS 
    WHERE TABLE_SCHEMA = 'dbo' 
      AND TABLE_NAME = 'DimStockItem' 
      AND COLUMN_NAME = 'Tags'
)
BEGIN
    ALTER TABLE DimStockItem DROP COLUMN Tags;
END
GO